In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Embedding, Dense, Dropout, Concatenate,
    MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
)
from keras.saving import register_keras_serializable

In [3]:
df = pd.read_excel("default of credit card clients.xls", skiprows=1)
df.rename(columns={"default payment next month": "target"}, inplace=True)
df.drop(columns=['ID'], inplace=True, errors='ignore')
df = df.apply(pd.to_numeric, errors='coerce')
df.fillna(df.median(numeric_only=True), inplace=True)

categorical_cols = ['SEX', 'EDUCATION', 'MARRIAGE']
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['target']]

le_dict = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le  

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

X_cat = df[categorical_cols].astype('int32').values 
X_num = df[numerical_cols].values                    
y = df['target'].astype(int).values


In [4]:
X_cat_train, X_cat_test, X_num_train, X_num_test, y_train, y_test = train_test_split(
    X_cat, X_num, y, test_size=0.2, random_state=42, stratify=y
)


In [5]:
checkpoint = ModelCheckpoint(
    filepath='best_tabtransformer_model.keras',
    monitor='val_accuracy',     
    save_best_only=True,
    save_weights_only=False,     
    verbose=1
)


In [6]:
@register_keras_serializable()
class TabTransformer(Model):
    def __init__(self, num_categories, num_numerical, emb_dim=8, num_heads=2, num_layers=2, dropout=0.3):
        super().__init__()
        self.embedding_layers = [
            Embedding(input_dim=n, output_dim=emb_dim) for n in num_categories
        ]

        self.attention_blocks = [
            [
                MultiHeadAttention(num_heads=num_heads, key_dim=emb_dim),
                LayerNormalization(),
                Dropout(dropout)
            ] for _ in range(num_layers)
        ]

        self.global_pool = GlobalAveragePooling1D()
        self.concat_dense = Dense(128, activation='relu')
        self.out_dense = Dense(1, activation='sigmoid')

    def call(self, inputs):
        cat_inputs, num_inputs = inputs
        x_cat = [emb(cat_inputs[:, i]) for i, emb in enumerate(self.embedding_layers)]
        x_cat = tf.stack(x_cat, axis=1)
        for mha, norm, drop in self.attention_blocks:
            attn_out = mha(x_cat, x_cat)
            x_cat = norm(x_cat + drop(attn_out))
        x_cat = self.global_pool(x_cat)
        x = Concatenate()([x_cat, num_inputs])
        x = self.concat_dense(x)
        return self.out_dense(x)


In [7]:
num_categories = [len(np.unique(X_cat[:, i])) for i in range(X_cat.shape[1])]
model = TabTransformer(num_categories=num_categories, num_numerical=X_num.shape[1])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    [X_cat_train, X_num_train], y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=64,
    callbacks=[checkpoint],
    verbose=1
)


Epoch 1/30
286/300 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7825 - loss: 0.4943
Epoch 1: val_accuracy improved from -inf to 0.80875, saving model to best_tabtransformer_model.keras
300/300 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7835 - loss: 0.4928 - val_accuracy: 0.8087 - val_loss: 0.4585
Epoch 2/30
286/300 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8139 - loss: 0.4416
Epoch 2: val_accuracy improved from 0.80875 to 0.81042, saving model to best_tabtransformer_model.keras
300/300 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8141 - loss: 0.4415 - val_accuracy: 0.8104 - val_loss: 0.4558
Epoch 3/30
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8199 - loss: 0.4360
Epoch 3: val_accuracy improved from 0.81042 to 0.81271, saving model to best_tabtransformer_model.keras
300/300 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8199 - loss: 0.4360 - val_accuracy: 0.8127 - val_loss: 0.4535
Epoch 4/30
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8163 - loss: 0.4

In [ ]:
y_pred_prob = model.predict([X_cat_test, X_num_test]).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("📈 F1 Score:", f1_score(y_test, y_pred))
print("🏆 ROC AUC:", roc_auc_score(y_test, y_pred_prob))
print("\n📄 Classification Report:\n", classification_report(y_test, y_pred))


188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
✅ Accuracy: 0.814
📈 F1 Score: 0.4571984435797665
🏆 ROC AUC: 0.7692434097271262

📄 Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.94      0.89      4673
           1       0.64      0.35      0.46      1327

    accuracy                           0.81      6000
   macro avg       0.74      0.65      0.67      6000
weighted avg       0.79      0.81      0.79      6000

